# Observer

In [ ]:
class Object:

  def __init__(self):
    self.__observers = []

  def __repr__(self):
    return "Object"

  def register(self, observer):
    self.__observers.append(observer)

  # Método que notifica os observadores do objeto sobre
  # kwargs são parâmetros nomeados
  def notify_all(self, *args, **kwargs):
    for observer in self.__observers:
      observer.notify(self, *args, **kwargs)

class ObserverA:

  def __init__(self, object):
    object.register(self)

  def notify(self, object, *args):
    print(f"The {type(self).__name__} received a {args[0]} from {object}")

class ObserverB:

  def __init__(self, object):
    object.register(self)

  def notify(self, object, *args):
    print(f"The {type(self).__name__} received a {args[0]} from {object}")

obj = Object()

obs_a = ObserverA(obj)
obs_b = ObserverB(obj)

obj.notify_all("Notification")

The ObserverA received a Notification from Object
The ObserverB received a Notification from Object


# Observer - Pull Model
Contexto: Agencia de notícias que possui diferentes tipos de inscritos(pessoas que recebem as notícias) e diferentes meios(ex: sms, email, etc). Quando a agencia vai publicar uma notícia ela deve atingir todos os meios e inscritos para essa nova notícia, para isso, será utilizado o padrão Observer.

In [ ]:
from abc import ABCMeta, abstractmethod

# Assunto/Tópico
class NewsAgency:

  # Construtor com os inscritos e última notícia
  def __init__(self):
    self.__registered = []
    self.__last_new = None

  # Método que recebe um inscrito e adiciona na lista
  def register(self, registered):
    self.__registered.append(registered)

  def unregister(self, registered = None):
    # Se não tiver inscrito específico ele tira o último
    if not registered:
      return self.__registered.pop()
    # Caso contrário, tira o inscrito especificado
    else:
      return self.__registered.remove(registered)

  # Método para mostrar os inscritos
  def registered(self):
    return [type(value).__name__ for value in self.__registered]

  # Método para notificar todos os inscritos
  def notify_all(self):
    for i in self.__registered:
      i.notify()

  # Método para adicionar nova notícia
  def add_news(self, news):
    self.__last_new = news

  # Método para mostrar nova notícia
  def show_news(self):
    return (f"LAST NEW!!! {self.__last_new}")

# Interface Observer
class RegisterType(metaclass=ABCMeta):

  @abstractmethod
  def notify(self):
    pass

# ObserverA
class SMSRegistered(RegisterType):

  def __init__(self, news_agency):
    self.news_agency = news_agency
    self.news_agency.register(self)

  def notify(self):
    print(f"{type(self).__name__}: {self.news_agency.show_news()}")

# ObserverB
class EmailRegistered(RegisterType):

  def __init__(self, news_agency):
    self.news_agency = news_agency
    self.news_agency.register(self)

  def notify(self):
    print(f"{type(self).__name__}: {self.news_agency.show_news()}")

# ObserverC
class DiscordRegistered(RegisterType):

  def __init__(self, news_agency):
    self.news_agency = news_agency
    self.news_agency.register(self)

  def notify(self):
    print(f"{type(self).__name__}: {self.news_agency.show_news()}")

# Client
if __name__ == "__main__":
  news_agency = NewsAgency()

  SMSRegistered(news_agency)
  EmailRegistered(news_agency)
  DiscordRegistered(news_agency)

  print(f"Registered: {news_agency.registered()}\n")

  news_agency.add_news("New pandemic growing at Brazil!")
  news_agency.notify_all()

  print(f"\nUnregistered: {type(news_agency.unregister()).__name__}\n")
  print(f"Registered: {news_agency.registered()}\n")

  news_agency.add_news("Victor is the new Design Patterns King!")
  news_agency.notify_all()

Registered: ['SMSRegistered', 'EmailRegistered', 'DiscordRegistered']

SMSRegistered: LAST NEW!!! New pandemic growing at Brazil!
EmailRegistered: LAST NEW!!! New pandemic growing at Brazil!
DiscordRegistered: LAST NEW!!! New pandemic growing at Brazil!

Unregistered: DiscordRegistered

Registered: ['SMSRegistered', 'EmailRegistered']

SMSRegistered: LAST NEW!!! Victor is the new Design Patterns King!
EmailRegistered: LAST NEW!!! Victor is the new Design Patterns King!


# Command

In [ ]:
s# Command
class Installer:

  def __init__(self, source, destiny):
    self.options = []
    self.source = source
    self.destiny = destiny

  def settings(self, choice):
    self.options.append(choice)

  def execute(self):
    for option in self.options:
      if list(option.values())[0]:
        print(f"Copying binaries from {self.source} to {self.destiny}")
      else:
        print("Operation has finished!")

# Client
if __name__ == "__main__":
  # Inicia o instalador
  installer = Installer("python-3.9.1.gzip", "/usr/bin/")

  # O usuário escolhe instalar apenas o Python
  installer.settings({"python": True})
  installer.settings({"java": False})

  # Executa a instalação
  installer.execute()

Copying binaries from python-3.9.1.gzip to /usr/bin/
Operation has finished!


In [ ]:
from abc import ABCMeta, abstractmethod

# Command
class Command(metaclass=ABCMeta):

  # Constructor para especificar o receiver
  def __init__(self, recv):
    self.recv = recv

  # Método abstrato
  @abstractmethod
  def execute(self):
    pass

# Concrete Command(Interface)
class ConcreteCommand(Command):

  def __init__(self, recv):
    self.recv = recv

  def execute(self):
    self.recv.action()

# Receiver
class Receiver:

  # Método que executa uma ação
  def action(self):
    print("Receiver Action!")

# Invoker
class Invoker:

  # Define o comando concreto e permite o uso de seus métodos
  def command(self, cmd):
    self.cmd = cmd

  # Método que utiliza o método execute do comando concreto
  def execute(self):
    self.cmd.execute()

# Client
if __name__ == "__main__":
  recv = Receiver()
  cmd = ConcreteCommand(recv)

  invoker = Invoker()
  invoker.command(cmd)
  invoker.execute()

Receiver Action!


# Command
Contexto: Em uma bolsa de valores, pessoas compram e vendem ações de empresas. Como usuário, é necessário criar pedidos(ordem de compra e venda) de ações. De forma que o corretor/agente execute esses pedidos criados pelo usuário(investidor).

In [ ]:
from abc import ABCMeta, abstractmethod

# Command Interface
class Order(metaclass=ABCMeta):

  @abstractmethod
  def execute(self):
    pass

# Concrete Commands
class BuyOrder(Order):

  def __init__(self, stocks):
    self.stocks = stocks

  def execute(self):
    self.stocks.buy()

class SellOrder(Order):

  def __init__(self, stocks):
    self.stocks = stocks

  def execute(self):
    self.stocks.sell()

# Receiver
class Stocks:

  def buy(self):
    print("You'll buy stocks!")

  def sell(self):
    print("You'll sell stocks!")

# Invoker
class Agent:

  def __init__(self):
    self.__order_queue = []

  def add_order_to_queue(self, order):
    self.__order_queue.append(order)
    order.execute()


if __name__ == "__main__":

  # Client
  stocks = Stocks()
  buy_order = BuyOrder(stocks)
  sell_order = SellOrder(stocks)

  # Invoker
  agent = Agent()
  agent.add_order_to_queue(buy_order)
  agent.add_order_to_queue(sell_order)

You'll buy stocks!
You'll sell stocks!


# Template Method

In [ ]:
from abc import ABCMeta, abstractmethod

# Interface
class Compiler(metaclass=ABCMeta):

  # Definindo métodos abstratos que as classes filhas vão implementar
  @abstractmethod
  def collect_font(self):
    pass

  @abstractmethod
  def compile_object(self):
    pass

  @abstractmethod
  def execute(self):
    pass

  # Template Method(possui o passo a passo da execução do algoritmo para cada subclasse)
  def compile_and_execute(self):
    self.collect_font()
    self.compile_object()
    self.execute()

# Subclasse
class IosCompiler(Compiler):

  def collect_font(self):
    print("Collecting Swift code")

  def compile_object(self):
    print("Compiling Swift code to LLVM bytecode...")

  def execute(self):
    print("Running on development environment!")

# Subclasse
class AndroidCompiler(Compiler):

  def collect_font(self):
    print("Collecting Kotlin code")

  def compile_object(self):
    print("Compiling Kotlin code to JVM bytecode...")

  def execute(self):
    print("Running on development environment!")

# Instanciando os objetos
ios = IosCompiler()
ios.compile_and_execute()

android = AndroidCompiler()
android.compile_and_execute()

Compiling Swift code to LLVM bytecode...
Running on development environment!
Compiling Kotlin code to JVM bytecode...
Running on development environment!


# Template Method

In [ ]:
from abc import ABCMeta, abstractmethod

class AbstractClass(metaclass=ABCMeta):

  def __init__(self):
    pass

  @abstractmethod
  def operation1(self):
    pass

  @abstractmethod
  def operation2(self):
    pass

  def template_method(self):
    print("Defining algorithm: Operation 1 after Operation 2")
    self.operation2()
    self.operation1()

class ConcreteClass(AbstractClass):

  def operation1(self):
    print("Concrete operation 1")

  def operation2(self):
    print("Concrete operation 2")

class Client:

  def main(self):
    self.concrete = ConcreteClass()
    self.concrete.template_method()

client = Client()
client.main()

Defining algorithm: Operation 1 after Operation 2
Concrete operation 2
Concrete operation 1


# Template Method
Contexto: Imagine que é dono de uma agência de viagens que oferece pacotes de viagens de 3 dias em diferentes lugares. O pacote inclui a viagem de ida, 3 dias no local escolhido e a viagem de volta.

In [ ]:
from abc import ABCMeta, abstractmethod

# Abstract Class
class Travel(metaclass=ABCMeta):

  @abstractmethod
  def going(self):
    pass

  @abstractmethod
  def day1(self):
    pass

  @abstractmethod
  def day2(self):
    pass

  @abstractmethod
  def day3(self):
    pass

  @abstractmethod
  def returning(self):
    pass

  # Template Method
  def timeline(self):
    self.going()
    self.day1()
    self.day2()
    self.day3()
    self.returning()

# Concrete Class
class BrazilTrip(Travel):

  def going(self):
    print("Airplane trip")

  def day1(self):
    print("Visiting Cristo Redentor")

  def day2(self):
    print("Going to the beach")

  def day3(self):
    print("Visiting Museum and eating feijoada")

  def returning(self):
    print("Airplane trip")


class UnitedStatesTrip(Travel):

  def going(self):
    print("Airplane trip")

  def day1(self):
    print("Visiting Statue of Liberty")

  def day2(self):
    print("Visiting World Trade Center")

  def day3(self):
    print("Visiting Broadway")

  def returning(self):
    print("Airplane trip")

class Agency:

  def prepare_trip(self):
    option = input("Which place you wanna go? [Brazil, United States]:")

    if option == "Brazil":
      self.trip = BrazilTrip()
      self.trip.timeline()

    elif option == "United States":
      self.trip = UnitedStatesTrip()
      self.trip.timeline()

    else:
      print("At the moment we don't have this place as an option!")

agency = Agency()
agency.prepare_trip()

Which place you wanna go? [Brazil, United States]:Brazil
Airplane trip
Visiting Cristo Redentor
Going to the beach
Visiting Museum and eating feijoada
Airplane trip


# State

In [ ]:
from abc import ABCMeta, abstractmethod

# State
class State(metaclass=ABCMeta):

  @abstractmethod
  def manipulate(self):
    pass

# Concrete States
class ConcreteStateA(State):

  def manipulate(self):
    print("Concrete State A!")

class ConcreteStateB(State):

  def manipulate(self):
    print("Concrete State B!")

# Context
class Context(State):

  # Construtor: Define o estado inicial como None
  def __init__(self):
    self.state = None

  # Método: Pega o estado
  def get_state(self):
    return self.state

  # Método: Define um novo estado
  def set_state(self, state):
    self.state = state

  # Método: Manipula o estado
  def manipulate(self):
    self.state.manipulate()

context = Context()
state_a = ConcreteStateA()
state_b = ConcreteStateB()

context.set_state(state_a)
context.manipulate()
print("Changing state...")

context.set_state(state_b)
context.manipulate()

Concrete State A!
Changing state...
Concrete State B!


# State
Contexto: Imagine que é possível redefinir o estado de um computador para ligado e desligado. Se estiver ligado, é possível suspender ou hibernar o computador. Para isso várias condições existem(e. g. Um computador suspenso não pode hibernar)

In [ ]:
from abc import ABCMeta

# State
class ComputerState(metaclass=ABCMeta):

  # Atributos de classe
  name = "State"
  allowed = []

  # Método para mudança de estados
  def change(self, state):
    if state.name in self.allowed:
      print(f"Current: {self} => Changed to a new state: {state.name}")
      self.__class__ = state
    else:
      print(f"Current: {self} => Is not possible to change to: {state.name}")

  def __str__(self):
    return self.name

# Concrete States
class TurnOn(ComputerState):
  name = "Turn On"
  allowed = ["Turn Off", "Suspend", "Hibernate"]

class TurnOff(ComputerState):
  name = "Turn Off"
  allowed = ["Turn On"]

class Suspend(ComputerState):
  name = "Suspend"
  allowed = ["Turn On"]

class Hibernate(ComputerState):
  name = "Hibernate"
  allowed = ["Turn On"]

# Context
class Computer:

  def __init__(self, model = "Dell XPS"):
    self.model = model
    self.state = TurnOff()

  def modify(self, state):
    self.state.change(state)

if __name__ == "__main__":
  computer = Computer()

  # Ligar
  computer.modify(TurnOn)

  # Desligar
  computer.modify(TurnOff)

  # Ligar novamente
  computer.modify(TurnOn)

  # Suspender
  computer.modify(Suspend)

  # Tentar hibernar(não pode)
  computer.modify(Hibernate)

  # Ligar novamente
  computer.modify(TurnOn)

  # Hibernar
  computer.modify(Hibernate)

Current: Turn Off => Changed to a new state: Turn On
Current: Turn On => Changed to a new state: Turn Off
Current: Turn Off => Changed to a new state: Turn On
Current: Turn On => Changed to a new state: Suspend
Current: Suspend => Is not possible to change to: Hibernate
Current: Suspend => Changed to a new state: Turn On
Current: Turn On => Changed to a new state: Hibernate
